In [1]:
from utils.BackTest import Engine, Strategy
import pandas as pd

In [2]:
df = pd.read_csv(r'../data/processed/combined_df_with_volatility.csv')
if 'Unnamed: 0' in df.columns:
    df = df.drop(columns=['Unnamed: 0'])

In [3]:
df

,datetime,reddit_smoothed_sentiment_weibull,news_smoothed_sentiment_weibull,tele_smoothed_sentiment_weibull,OPEN,HIGH,LOW,CLOSE,TOTAL_TRADES,TOTAL_TRADES_BUY,...,H-L,H-PC,L-PC,TR,ATR,log_return_100,predicted_conditional_volatility,rolling_mean_volatility,rolling_std_volatility,volatility_category
0,2024-03-23 18:00:00+00:00,-0.000662,232.686671,0.000000,65652.70,65681.72,65270.01,65373.88,56859,25413,...,411.71,29.02,382.69,411.71,658.445000,-0.425594,0.005914,0.006518,0.001199,Normal
1,2024-03-23 19:00:00+00:00,-0.000241,243.310915,0.000000,65373.88,65485.15,64786.25,64872.90,73300,32361,...,698.90,111.27,587.63,698.90,701.053333,-0.769282,0.005821,0.006418,0.001168,Normal
2,2024-03-23 20:00:00+00:00,-0.000083,263.874155,0.000000,64872.90,65118.20,64672.93,65009.78,55912,27474,...,445.27,245.30,199.97,445.27,604.058333,0.210775,0.006339,0.006296,0.001028,Normal
3,2024-03-23 21:00:00+00:00,-0.000027,265.004313,0.000000,65009.78,65199.97,64780.00,64944.69,45367,21035,...,419.97,190.19,229.78,419.97,571.245000,-0.100174,0.005951,0.006184,0.000926,Normal
4,2024-03-23 22:00:00+00:00,-0.000008,248.893742,0.000000,64944.69,65050.72,64744.55,64747.86,46105,21611,...,306.17,106.03,200.14,306.17,468.430000,-0.303533,0.005584,0.006049,0.000790,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7405,2025-01-26 07:00:00+00:00,0.055311,-13.564028,0.000000,104990.75,105100.00,104950.00,104998.35,41775,22248,...,150.00,109.25,40.75,150.00,298.715000,0.007238,0.003691,0.003776,0.000097,Normal
7406,2025-01-26 08:00:00+00:00,0.030398,-6.781369,0.103237,104998.35,105221.36,104863.31,105030.40,63281,30294,...,358.05,223.01,135.04,358.05,325.368333,0.030520,0.003626,0.003759,0.000091,Low
7407,2025-01-26 09:00:00+00:00,0.015620,0.681248,0.000000,105030.40,105030.41,104444.01,104630.03,91103,41647,...,586.40,0.01,586.39,586.40,326.935000,-0.381923,0.003572,0.003744,0.000097,Low
7408,2025-01-26 10:00:00+00:00,0.007520,8.190720,0.000000,104630.03,104722.90,104500.00,104715.29,50517,26960,...,222.90,92.87,130.03,222.90,277.221667,0.081454,0.003957,0.003754,0.000108,High


In [4]:
df['datetime'] = pd.to_datetime(df['datetime']).dt.tz_localize(None)
df = df.set_index('datetime')

In [5]:
df['typical_price'] = (df['HIGH'] + df['LOW'] + df['CLOSE']) / 3
#df['typical_price'] = df['typical_price'].shift(1) Don't shift as we make decision at the end of each bar, so we have the latest information

In [6]:
#Daily VWAP (UTC-based)
#Reset VWAP every UTC day. This is the most common standard for crypto backtests and analysis.
df['DATE'] = df.index.date  
df['VWAP'] = df.groupby('DATE').apply(
    lambda g: (g['typical_price'] * g['VOLUME']).cumsum() / g['VOLUME'].cumsum()
).reset_index(level=0, drop=True)


# #Rolling VWAP (24-hour window)
# rolling_window = 24  # if hourly bars
# vwap_numerator = (df['typical_price'] * df['VOLUME']).rolling(rolling_window).sum()
# vwap_denominator = df['VOLUME'].rolling(rolling_window).sum()
# df['VWAP'] = vwap_numerator / vwap_denominator

C:\Users\leoqi\AppData\Local\Temp\ipykernel_60520\68637714.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df['VWAP'] = df.groupby('DATE').apply(


In [7]:
#VWAP strategy
def determine_signal(df):
    """
    Determine the trading signal based on the strategy's criteria.
    """
    if df['CLOSE'] > df['VWAP']:
        return 1
    elif df['CLOSE'] < df['VWAP']:
        return 0
    else:
        return None


In [8]:
df['VWAP_signal'] = df.apply(determine_signal, axis=1)

df['VWAP_signal'] = df['VWAP_signal'].fillna(0)

In [9]:
df_filtered = df.rename(columns={'OPEN':'Open',
                                 'CLOSE':'Close',
                                 'HIGH':'High',
                                 'LOW':'Low'})

In [10]:
# class VWAP_Strategy(Strategy):
#     def on_bar(self):
#         if self.data.loc[self.current_idx]['VWAP_signal'] == 1:
#             self.buy(
#                 "btc",
#                size = round(0.20 * (self.cash + (self.position_size() * self.close())) / self.close(), 8)
# #                size = round((0.10 * self.cash)/ self.close(), 8)
#             )

In [11]:
class VWAP_Strategy(Strategy):
    def on_bar(self):
        # If the current index is the first one in the data, buy in 80% of available cash (after transaction cost)
        if self.current_idx == self.data.index[0]:  # or self.data.index[0] if that's a datetime index
            # Buy in 80% of available cash after subtracting 1% for transaction cost
            size = round((0.80 * (self.cash - (0.001 * self.cash))) / self.close(), 8)
            self.buy("btc", size=size)

        elif(self.data.loc[self.current_idx]['VWAP_signal'] == 1):
            size = round(0.20 * (self.cash + (self.position_size() * self.close())) / self.close(), 8)
            self.buy("btc", size=size)
        
        # # If VWAP_signal is 1, perform buy action based on the strategy
        # elif(self.data.loc[self.current_idx]['VWAP_signal'] == 1) & (self.data.loc[self.current_idx - pd.Timedelta(hours=12)]['VWAP_signal'] == 1):
        #     size = round(0.20 * (self.cash + (self.position_size() * self.close())) / self.close(), 8)
        #     self.buy("btc", size=size)

        # elif (self.data.loc[self.current_idx]['VWAP_signal'] == 1) & (self.data.loc[self.current_idx]['news_smoothed_sentiment_weibull'] < -0.5):
        #     size = round(0.10 * (self.cash + (self.position_size() * self.close())) / self.close(), 8)
        #     self.sell("btc", size=size)

        # elif (self.data.loc[self.current_idx]['VWAP_signal'] == 0) & (self.data.loc[self.current_idx]['news_smoothed_sentiment_weibull'] > 0.5):
        #     size = round(0.10 * (self.cash + (self.position_size() * self.close())) / self.close(), 8)
        #     self.buy("btc", size=size)

        # elif (self.data.loc[self.current_idx]['VWAP_signal'] == 0) & (self.data.loc[self.current_idx]['news_smoothed_sentiment_weibull'] < -0.3):
        #     size = round(0.10 * (self.cash + (self.position_size() * self.close())) / self.close(), 8)
        #     self.sell("btc", size=size)


In [12]:
class BB_RSI_Strategy(Strategy):
    def on_bar(self):
        # Get the current BB_RSI signal
        bb_rsi_signal = self.data.loc[self.current_idx]['BB_RSI_Signal']
        
        # Logic to decide on trading action
        # Buy condition: When VWAP signal is 1 and BB_RSI Signal is 1 (oversold + below lower BB + RSI < 30)
        if bb_rsi_signal == 1:
            self.buy(
                "btc",
                size=round(0.10 * (self.cash + (self.position_size() * self.close())) / self.close(), 8)
            )

In [13]:
class EMA_Strategy(Strategy):
    def on_bar(self):
        # Get the current BB_RSI signal
        EMA_Signal = self.data.loc[self.current_idx]['EMA_Signal']
        
        # Logic to decide on trading action
        # Buy condition: When VWAP signal is 1 and BB_RSI Signal is 1 (oversold + below lower BB + RSI < 30)
        if EMA_Signal == 1:
            self.buy(
                "btc",
                size=round(0.10 * (self.cash + (self.position_size() * self.close())) / self.close(), 8)
            )

In [14]:
class VWAP_BB_RSI_Strategy(Strategy):
    def on_bar(self):
        # Get the current VWAP signal
        vwap_signal = self.data.loc[self.current_idx]['VWAP_signal']
        
        # Get the current BB_RSI signal
        bb_rsi_signal = self.data.loc[self.current_idx]['BB_RSI_Signal']
        
        # Logic to decide on trading action
        # Buy condition: When VWAP signal is 1 and BB_RSI Signal is 1 (oversold + below lower BB + RSI < 30)
        if vwap_signal == 1 and bb_rsi_signal == 1:
            self.buy(
                "btc",
                size=round(0.20 * (self.cash + (self.position_size() * self.close())) / self.close(), 8)
            )

In [15]:
validate_frame = df_filtered[df_filtered.index >= pd.to_datetime('2025-01-01').tz_localize(None)]

In [16]:
validate_frame.isna().sum()

reddit_smoothed_sentiment_weibull    0
news_smoothed_sentiment_weibull      0
tele_smoothed_sentiment_weibull      0
Open                                 0
High                                 0
Low                                  0
Close                                0
TOTAL_TRADES                         0
TOTAL_TRADES_BUY                     0
TOTAL_TRADES_SELL                    0
VOLUME                               0
VOLUME_BUY                           0
VOLUME_SELL                          0
return                               0
log_return                           0
hourly_volatility                    0
EMA_8                                0
EMA_13                               0
EMA_21                               0
EMA_Signal                           0
EMA_short                            0
EMA_long                             0
MACD                                 0
Signal_Line                          0
MACD_Signal                          0
MACD_Hist                

In [17]:
validate_frame['RSI_14'].tail(50)

datetime
2025-01-24 10:00:00    68.905952
2025-01-24 11:00:00    71.528204
2025-01-24 12:00:00    66.664657
2025-01-24 13:00:00    62.116387
2025-01-24 14:00:00    72.396746
2025-01-24 15:00:00    75.998170
2025-01-24 16:00:00    76.061432
2025-01-24 17:00:00    75.757922
2025-01-24 18:00:00    69.389211
2025-01-24 19:00:00    56.193715
2025-01-24 20:00:00    48.686302
2025-01-24 21:00:00    50.743897
2025-01-24 22:00:00    42.939010
2025-01-24 23:00:00    47.160176
2025-01-25 00:00:00    40.113973
2025-01-25 01:00:00    43.635156
2025-01-25 02:00:00    41.095921
2025-01-25 03:00:00    50.744351
2025-01-25 04:00:00    40.052230
2025-01-25 05:00:00    37.029339
2025-01-25 06:00:00    38.061099
2025-01-25 07:00:00    28.715328
2025-01-25 08:00:00    29.236219
2025-01-25 09:00:00    33.556153
2025-01-25 10:00:00    45.455762
2025-01-25 11:00:00    43.282968
2025-01-25 12:00:00    49.611424
2025-01-25 13:00:00    47.487672
2025-01-25 14:00:00    53.316227
2025-01-25 15:00:00    52.959480
2

In [18]:
validate_frame[['BB_RSI_Signal', 'VWAP_signal']].tail(50)

,BB_RSI_Signal,VWAP_signal
datetime,,
2025-01-24 10:00:00,0,1
2025-01-24 11:00:00,0,1
2025-01-24 12:00:00,0,1
2025-01-24 13:00:00,0,1
2025-01-24 14:00:00,0,1
2025-01-24 15:00:00,0,1
2025-01-24 16:00:00,0,1
2025-01-24 17:00:00,0,1
2025-01-24 18:00:00,0,1


In [19]:
morning_session = validate_frame.between_time('00:00', '11:59')  # 00:00 UTC to 11:59 UTC
afternoon_session = validate_frame.between_time('12:00', '23:59')  # 12:00 UTC to 23:59 UTC


morning_session_resampled = morning_session.reset_index().groupby('DATE').last().set_index('datetime')
afternoon_session_resampled = afternoon_session.reset_index().groupby('DATE').last().set_index('datetime')

validate_frame_half_day = pd.concat([morning_session_resampled, afternoon_session_resampled], axis=0)

In [20]:
validate_frame_half_day = validate_frame_half_day.sort_index()

In [21]:
validate_frame_half_day

,reddit_smoothed_sentiment_weibull,news_smoothed_sentiment_weibull,tele_smoothed_sentiment_weibull,Open,High,Low,Close,TOTAL_TRADES,TOTAL_TRADES_BUY,TOTAL_TRADES_SELL,...,TR,ATR,log_return_100,predicted_conditional_volatility,rolling_mean_volatility,rolling_std_volatility,volatility_category,typical_price,VWAP,VWAP_signal
datetime,,,,,,,,,,,,,,,,,,,,,
2025-01-01 11:00:00,-2.079378e+00,-4.262254,0.000000,93326.23,93705.08,93200.00,93444.64,58142,29479,28663,...,505.08,361.283333,0.126797,0.004374,0.005334,0.000457,Low,93449.906667,93681.946380,0
2025-01-01 23:00:00,1.963687e+00,251.557137,0.720349,94802.98,94952.29,94542.00,94591.79,68059,31848,36211,...,410.29,429.560000,-0.223016,0.003940,0.004496,0.000494,Low,94695.360000,94055.346447,1
2025-01-02 11:00:00,6.707777e-02,-29.570431,0.000000,96708.00,96876.38,96469.46,96732.99,64369,32908,31461,...,406.92,450.438333,0.025837,0.004519,0.004298,0.000252,Normal,96692.943333,95706.178910,1
2025-01-02 23:00:00,4.559581e-07,-157.255780,0.000000,96911.04,97077.78,96758.17,96984.79,86210,45531,40679,...,319.61,541.291667,0.076072,0.004782,0.004854,0.000556,Normal,96940.246667,96430.860654,1
2025-01-03 11:00:00,8.841680e-16,-196.097927,0.000000,96488.53,96830.00,96482.51,96757.58,67414,35654,31760,...,347.49,396.815000,0.278453,0.003848,0.004585,0.000753,Normal,96690.030000,96653.017521,1
2025-01-03 23:00:00,1.225550e-01,-131.761843,0.000000,98296.37,98382.19,98067.99,98174.18,63505,24976,38529,...,314.20,405.093333,-0.124385,0.003818,0.003957,0.000185,Normal,98208.120000,97437.677124,1
2025-01-04 11:00:00,-9.845882e-02,-54.634368,0.000000,97789.32,97923.07,97711.54,97889.39,42774,25957,16817,...,211.53,290.695000,0.102280,0.003404,0.003765,0.000319,Low,97841.333333,97998.960636,0
2025-01-04 23:00:00,2.706029e+00,0.000000,0.000000,98417.25,98419.14,98143.20,98220.50,48797,24404,24393,...,275.94,347.021667,-0.200114,0.003337,0.003345,0.000147,Normal,98260.946667,98030.771188,1
2025-01-05 11:00:00,-1.241244e+00,-7.526729,0.000000,97737.48,97822.16,97615.13,97641.63,45553,26172,19381,...,207.03,261.843333,-0.098117,0.002818,0.003080,0.000253,Low,97692.973333,98058.753324,0


In [22]:
validate_frame_half_day.loc['2025-01-13',:]

,reddit_smoothed_sentiment_weibull,news_smoothed_sentiment_weibull,tele_smoothed_sentiment_weibull,Open,High,Low,Close,TOTAL_TRADES,TOTAL_TRADES_BUY,TOTAL_TRADES_SELL,...,TR,ATR,log_return_100,predicted_conditional_volatility,rolling_mean_volatility,rolling_std_volatility,volatility_category,typical_price,VWAP,VWAP_signal
datetime,,,,,,,,,,,,,,,,,,,,,
2025-01-13 11:00:00,1.339147,-184.931673,0.0,91644.96,91776.04,90833.12,90851.22,341718,147524,194194,...,942.92,882.223333,-0.869876,0.008091,0.004760,0.001697,High,91153.460000,92826.834072,0
2025-01-13 23:00:00,2.179069,-117.145261,0.0,94399.98,94612.70,94164.77,94536.10,93177,44956,48221,...,447.93,952.300000,0.144091,0.007174,0.006498,0.001259,Normal,94437.856667,92242.324430,1


In [23]:
validate_frame_half_day[['hourly_volatility', 'predicted_conditional_volatility','volatility_category']].tail(50)

,hourly_volatility,predicted_conditional_volatility,volatility_category
datetime,,,
2025-01-01 23:00:00,0.002354,0.003940,Low
2025-01-02 11:00:00,0.003471,0.004519,Normal
2025-01-02 23:00:00,0.005105,0.004782,Normal
2025-01-03 11:00:00,0.002240,0.003848,Normal
2025-01-03 23:00:00,0.002906,0.003818,Normal
2025-01-04 11:00:00,0.001965,0.003404,Low
2025-01-04 23:00:00,0.002327,0.003337,Normal
2025-01-05 11:00:00,0.001652,0.002818,Low
2025-01-05 23:00:00,0.002006,0.002602,Normal


In [24]:
e = Engine(initial_cash = 100_000,asset_type='cryptocurrencies')
e.add_data(validate_frame)
e.add_strategy(VWAP_Strategy()) # Pass both df_filtered and strategy_dict
stats = e.run()

 32%|███▏      | 195/612 [00:00<00:00, 991.21it/s]

2025-01-01 01:00:00 New position opened: <Position: btc size: 0.84659995 entry: 94401.14>,transaction fee: 79.92000040394299
2025-01-01 03:00:00 Buy Entry Not Filled. Insufficient cash.
2025-01-01 08:00:00 Stop Loss Filled. Limit: 93416.82 / Low: 93304.0
2025-01-01 08:00:00 Position closed for ticker btc at 93416.82, transaction fee: 79.086675141159
2025-01-01 13:00:00 New position opened: <Position: btc size: 0.21317244 entry: 93820.76>,transaction fee: 20.000000331854398
2025-01-01 14:00:00 New position opened: <Position: btc size: 0.25532858 entry: 94031.76>,transaction fee: 24.008995755700802
2025-01-01 15:00:00 New position opened: <Position: btc size: 0.30606878 entry: 94175.89>,transaction fee: 28.824299757714197
2025-01-01 16:00:00 Buy Entry Not Filled. Insufficient cash.
2025-01-01 17:00:00 Buy Entry Not Filled. Insufficient cash.
2025-01-01 18:00:00 Buy Entry Not Filled. Insufficient cash.
2025-01-01 19:00:00 Buy Entry Not Filled. Insufficient cash.
2025-01-01 20:00:00 Buy En

 63%|██████▎   | 383/612 [00:00<00:00, 829.51it/s]

2025-01-10 18:00:00 Buy Entry Not Filled. Insufficient cash.
2025-01-10 19:00:00 Buy Entry Not Filled. Insufficient cash.
2025-01-10 20:00:00 Buy Entry Not Filled. Insufficient cash.
2025-01-10 21:00:00 Buy Entry Not Filled. Insufficient cash.
2025-01-10 22:00:00 Buy Entry Not Filled. Insufficient cash.
2025-01-10 23:00:00 Buy Entry Not Filled. Insufficient cash.
2025-01-11 00:00:00 Buy Entry Not Filled. Insufficient cash.
2025-01-11 01:00:00 Buy Entry Not Filled. Insufficient cash.
2025-01-11 06:00:00 Buy Entry Not Filled. Insufficient cash.
2025-01-11 10:00:00 Buy Entry Not Filled. Insufficient cash.
2025-01-11 11:00:00 Take Profit Filled. Limit: 94441.22 / High: 94791.67
2025-01-11 11:00:00 Position closed for ticker btc at 94441.22, transaction fee: 29.1771304355414
2025-01-11 11:00:00 New position opened: <Position: btc size: 0.36682564 entry: 94488.86>,transaction fee: 34.6609365423704
2025-01-11 12:00:00 Buy Entry Not Filled. Insufficient cash.
2025-01-11 13:00:00 Buy Entry Not 

 76%|███████▋  | 467/612 [00:00<00:00, 759.21it/s]

2025-01-16 23:00:00 Buy Entry Not Filled. Insufficient cash.
2025-01-17 00:00:00 Buy Entry Not Filled. Insufficient cash.
2025-01-17 02:00:00 Buy Entry Not Filled. Insufficient cash.
2025-01-17 05:00:00 Buy Entry Not Filled. Insufficient cash.
2025-01-17 06:00:00 Buy Entry Not Filled. Insufficient cash.
2025-01-17 07:00:00 Buy Entry Not Filled. Insufficient cash.
2025-01-17 08:00:00 Buy Entry Not Filled. Insufficient cash.
2025-01-17 09:00:00 Take Profit Filled. Limit: 101874.49 / High: 102288.0
2025-01-17 09:00:00 Position closed for ticker btc at 101874.49, transaction fee: 20.588191600968102
2025-01-17 09:00:00 Take Profit Filled. Limit: 101902.91 / High: 102288.0
2025-01-17 09:00:00 Position closed for ticker btc at 101902.91, transaction fee: 29.629868920128803
2025-01-17 09:00:00 New position opened: <Position: btc size: 0.34270047 entry: 102136.01>,transaction fee: 35.002058630924694
2025-01-17 10:00:00 Take Profit Filled. Limit: 102561.74 / High: 102581.45
2025-01-17 10:00:00 P

100%|██████████| 612/612 [00:00<00:00, 791.19it/s]

2025-01-23 14:00:00 New position opened: <Position: btc size: 0.23404308 entry: 102512.28>,transaction fee: 23.992289749022397
2025-01-23 15:00:00 New position opened: <Position: btc size: 0.27584279 entry: 105213.5>,transaction fee: 29.022385385665004
2025-01-23 16:00:00 New position opened: <Position: btc size: 0.33092102 entry: 105263.52>,transaction fee: 34.833911407190406
2025-01-23 17:00:00 Buy Entry Not Filled. Insufficient cash.
2025-01-23 18:00:00 Buy Entry Not Filled. Insufficient cash.
2025-01-23 19:00:00 Buy Entry Not Filled. Insufficient cash.
2025-01-23 23:00:00 Buy Entry Not Filled. Insufficient cash.
2025-01-24 04:00:00 Buy Entry Not Filled. Insufficient cash.
2025-01-24 05:00:00 Buy Entry Not Filled. Insufficient cash.
2025-01-24 06:00:00 Buy Entry Not Filled. Insufficient cash.
2025-01-24 07:00:00 Buy Entry Not Filled. Insufficient cash.
2025-01-24 08:00:00 Buy Entry Not Filled. Insufficient cash.
2025-01-24 09:00:00 Buy Entry Not Filled. Insufficient cash.
2025-01-24

In [25]:
e.plot(show_signals= True)